# 1. 기존의 user_game_data csv파일 전처리 (컬럼명 변환, 데이터 수 10000개로 축소)

In [ ]:
import pandas as pd

df = pd.read_csv('~/Project/user_game_data (2).csv')
df['userId'] = 1
df['score'] = round(df['elapsedTime'])*5 + df['numKilled']*10
df.rename(columns = {
    'userId' : 'user_id',
    'playedTime' : 'when_played',
    'elapsedTime': 'elapsed_time',
    'numKilled' : 'kill_count',
},inplace=True)
df_new = df.iloc[:10000]
df_new.to_csv('~/Project/user_game_data.csv',index=False)
df_new

# 2. Pycaret Forecasting을 통한 score 예측 모델 학습 

In [ ]:
!pip install pycaret

In [ ]:
# 필요 모듈 import , 데이터 불러오기
import pandas as pd
# from pycaret.regression import *
from pycaret.time_series import *
# csv 불러오기
df = pd.read_csv('~/Project/user_game_data.csv')
# when_played 컬럼 날짜 12시간 단위로 재배치 
start_date = '2010-09-22 12:00'
end_date = '2024-05-31 00:00'
date_range= pd.date_range(start=start_date,end=end_date,freq='12H')
df['when_played'] = date_range[:len(df)]

# Pycret automl을 사용하기위해 playedTime 속성을 index로 사용
user_data = df.set_index('when_played')

# 최소 데이터 포인트 수 설정 (예: 5)
if len(user_data) < 5:
  print(f"User ID: {user_id} does not have enough data points.")
  pass
# TSForecastingExperiment() 생성 
exp = TSForecastingExperiment()
exp.setup(data=user_data[['elapsed_time','kill_count','score']],
          target='score',
          session_id = '123', fh=1, fold = 5)
# 최적 모델 비교 및 최적 모델 선택
# setup()에서 설정한 데이터를 바탕으로 최적의 모델 산출
best_model = exp.compare_models(sort='MAE')
print(best_model)
# 최적 모델 훈련
final_model = exp.finalize_model(best_model)
# 학습 모델 저장 
_ = exp.save_model(final_model, "final_model")

# 3. 외생변수 예측하기

In [ ]:
# 1. 사용할 외생변수 추출
exog_vars = ['elapsedTime',	'numKilled']
data = user_data[exog_vars]
# 2 : 외생변수 각각에 대한 시계열 예측 수행
'''
주의할 점 -> 기존 user_data의 외래변수를 예측하는 과정 예측하기 전에 이전 시점에서 진행한 게임에 대한 데이터를 user_data에 행 추가를 
진행한 뒤에 외래변수 예측 
-> 영향이 많이 없을수도 있으나 이전 시점의 영향을 반영하기 위함 
'''
exog_exps = []
exog_models = []
# global_fig_settings = {'figsize': (10, 6)}
for exog_var in exog_vars:
    # 외생변수에 대한 예측을 도출하기 위하여 시계열 실험 생성
    exog_exp = TSForecastingExperiment()
    # setup
    exog_exp.setup(
        data=data[[exog_var]],
        target=exog_var,
        fh=1,
        session_id=42,
        verbose=False,
    )

    # 사용자는 미래 외생 변수를 모델링하는 방법을 사용자 정의할 수 있습니다.
    # 즉, 모델링 시간이 늘어나는 대신 더 나은 모델을 얻을 수 있는 추가 단계와 모델을 추가할 수 있습니다.
    best = exog_exp.compare_models(
        sort="mae",
        verbose=False,
    )
    final_exog_model = exog_exp.finalize_model(best)

    exog_exps.append(exog_exp)
    exog_models.append(final_exog_model)
    
# 3: 외생 변수에 대한 미래 예측 얻기
future_exog = [exog_exp.predict_model(exog_model) for exog_exp, exog_model in zip(exog_exps, exog_models)]
#print(future_exog[0])   # elapsedTime 예측값
#print(future_exog[1])   # numKilled 예측값 
# 4. 예측값 concat
future_exog = pd.concat(future_exog, axis=1)
future_exog.columns = exog_vars
future_exog

# 4. 모델을 로드하고 목표 변수에 대한 미래 예측하기 

In [ ]:
# 미래 예측용 시계열 실험 생성
exp_future = TSForecastingExperiment()
# 이전에 저장한 모델 로드
final_slim_model = exp_future.load_model("final_model")
# 최종 예측
future_preds = exp_future.predict_model(
    final_slim_model,  # 모델 입력
    X=future_exog,  # 외생변수 입력
)
future_preds